In [325]:
# Importing necessary libraries and packages
import pandas as pandas
import numpy as numpy
import matplotlib.pyplot
from IPython.display import Markdown, display
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.svm import SVC

In [326]:
# Loading the data from csv file and Separating table into values(x) and outcome(y)

data_table = pandas.read_csv('/Users/banks007/Desktop/Archive/historical_table_adjusted_2.csv', sep=',', header=0)
x = data_table.values[:,0:9]
y = data_table.values[:,10]

In [327]:
# Splitting data into training dataset and testing dataset: train (1457 rows), test (365 rows)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10, test_size = 0.2 )

In [328]:
# Applying Support vector machine (SVM) Classifier to Data

from sklearn import svm
svm_classifier = svm.SVC(kernel='linear', gamma='auto', C=2)
svm_classifier.fit(x_train, y_train)
svm_classifier.score(x_test, y_test)

0.7178082191780822

In [329]:
# Applying Random Forest Classifier to Data

rf = RandomForestClassifier(n_estimators=40)
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.9397260273972603

In [330]:
# Applying Naive Bayes Multinomial Classifier to Data

m_nb = MultinomialNB()
m_nb.fit(x_train, y_train)
m_nb.score(x_test, y_test)

0.5589041095890411

In [331]:
# Applying  Logistic Regression Classifier to Data 

lr = LogisticRegression(solver = 'liblinear', multi_class='ovr')
lr.fit(x_train, y_train)
lr.score(x_test, y_test)

0.6986301369863014

In [332]:
# Applying Decision Tree Classifier to data
    
dtc = DecisionTreeClassifier(criterion = "entropy", random_state=100, max_depth=3, min_samples_leaf=5)
dtc.fit(x_train, y_train)
dtc.score(x_test, y_test)

0.7315068493150685

In [333]:
# Applying K-Nearest Neighbor (KNN) Classifier to data
# preprocessing values for KNN
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test= sc_x.transform(x_test)

KNN_classifier = KNeighborsClassifier(n_neighbors=2, p=2, metric='euclidean')
KNN_classifier.fit(x_train, y_train)
KNN_classifier.score(x_train, y_train)

0.9450926561427591

### Implementing K - Fold on Data

K-Folds cross-validator

Provides train/test indices to split data in train/test sets. Split
dataset into k consecutive folds (without shuffling by default).

Each fold is then used once as a validation while the k - 1 remaining
folds form the training set. 

K-fold allows all the data to be used in training and testing of the data in other to allow the model to maximise learning and testing with all available data. This helps the model to learn better and give more accurate classification and prediction.

K- fold does a specified round of learning on the Data and in each round it changes the part of the data used in testing and training to maximise the learning of the patterns in the table and maximise prediction result
table is split into 20 parts and the correspoing parts are used for trainnig and testing <br>

|Round|  Testing|Training|
|-----|-------|------|
|1    | 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19     |  0 1 2 3 4     |
|2    | 0  1  2  3  4 10 11 12 13 14 15 16 17 18 19      |  5 6 7 8 9      |
|3    | 0  1  2  3  4  5  6  7  8  9 15 16 17 18 19     | 10 11 12 13 14     |
|4    | 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14      | 15 16 17 18 19     |

In [334]:
# implementing K-fold on dataset

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=4)
k_fold

KFold(n_splits=4, random_state=None, shuffle=False)

In [335]:
# creating indices for k-fold

for train_index, test_index in k_fold.split([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]):
    print(train_index, test_index)

[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2 3 4]
[ 0  1  2  3  4 10 11 12 13 14 15 16 17 18 19] [5 6 7 8 9]
[ 0  1  2  3  4  5  6  7  8  9 15 16 17 18 19] [10 11 12 13 14]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] [15 16 17 18 19]


### Creating a function to run supplied classifier/algorithm and return score
The function take classifier/model type and run it against the test and train data to get a score for the specific model

In [336]:
# Creating a function that takes specific model type and data, use data to learn and test to test model and return score

def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    
    return model.score(x_test, y_test)

In [337]:
# creating a sparate fucntion for KNN

def get_score_KNN(model, x_train, x_test, y_train, y_test):
#    sc_x = StandardScaler()
    x_train = sc_x.fit_transform(x_train)
    x_test = sc_x.transform(x_test)

    #KNN_classifier = KNeighborsClassifier()
    model.fit(x_train, y_train)
#    KNN_classifier.score(x_train, y_train)
    return model.score(x_test, y_test)

In [338]:
# Initializing array to store each models' score from the k-fold iterations
scores_svm = []
scores_KNN = []
scores_rf = []
scores_lr = []
scores_n_bayes = []
scores_dtc = []
scores_knnc = []

for train_index, test_index in k_fold.split(x):
    x_train, x_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    
    scores_svm.append(get_score(SVC(), x_train, x_test, y_train, y_test))
    scores_KNN.append(get_score_KNN(KNeighborsClassifier(n_neighbors=2, p=2, metric='euclidean'), x_train, x_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(), x_train, x_test, y_train, y_test))
    scores_lr.append(get_score(LogisticRegression(solver = 'liblinear', multi_class='ovr'), x_train, x_test, y_train, y_test))    
    scores_n_bayes.append(get_score(MultinomialNB(), x_train, x_test, y_train, y_test))
    scores_dtc.append(get_score(DecisionTreeClassifier(), x_train, x_test, y_train, y_test))

In [339]:
#scores_KNN
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)
KNN_classifier.score(x_train, y_train)

0.8961228968544257

In [340]:
get_score_KNN(KNeighborsClassifier(n_neighbors=2, p=2, metric='euclidean'), x_train, x_test, y_train, y_test)

0.7472527472527473

In [341]:
avg_svm = sum(scores_svm)/ len(scores_svm)
avg_n_bayes  = sum(scores_n_bayes)/ len(scores_n_bayes)
avg_lr = sum(scores_lr)/ len(scores_lr)
avg_KNN = sum(scores_KNN)/ len(scores_KNN)
avg_r_forest = sum(scores_rf)/ len(scores_rf)
avg_dtc = sum(scores_dtc)/ len(scores_dtc)

print ('SVM: {0} Avg = {1}'.format(scores_svm, avg_svm))
print ('NBA: {0} Avg = {1}'.format(scores_n_bayes, avg_n_bayes))
print ('LRA: {0} Avg = {1}'.format(scores_lr, avg_lr))
print ('RFT: {0} Avg = {1}'.format(scores_rf, avg_r_forest))
print ('DTC: {0} Avg = {1}'.format(scores_dtc, avg_dtc))
print ('KNN: {0} Avg = {1}'.format(scores_KNN, avg_KNN))

SVM: [0.4342105263157895, 0.5855263157894737, 0.589010989010989, 0.6021978021978022] Avg = 0.5527364083285137
NBA: [0.4342105263157895, 0.5855263157894737, 0.6527472527472528, 0.6945054945054945] Avg = 0.5917473973395027
LRA: [0.6359649122807017, 0.7039473684210527, 0.6263736263736264, 0.621978021978022] Avg = 0.6470659822633507
RFT: [0.8399122807017544, 0.8925438596491229, 0.9494505494505494, 0.8087912087912088] Avg = 0.8726744746481587
DTC: [0.8289473684210527, 0.881578947368421, 0.9384615384615385, 0.7868131868131868] Avg = 0.8589502602660497
KNN: [0.7653508771929824, 0.7982456140350878, 0.8813186813186813, 0.7472527472527473] Avg = 0.7980419799498747


### Printing Result of various classifier on Dataset

In [345]:
from sklearn.metrics import classification_report

y_predict_svm = svm_classifier.predict(x_test)
y_predict_KNN = KNN_classifier.predict(x_test)
y_predict_rf = rf.predict(x_test)
y_predict_m_nb = m_nb.predict(x_test)
y_predict_lr = lr.predict(x_test)
y_predict_dtc = dtc.predict(x_test)


print('')
display(Markdown('**Support Vector Machine Classifier**'))
print(classification_report(y_test, y_predict_svm))
print("Accuracy is = ", accuracy_score(y_test, y_predict_svm)*100)
print('')
print('')

display(Markdown('**Naive Bayes - Multinomial Classifier**'))
print(classification_report(y_test, y_predict_m_nb))
print("Accuracy is = ", accuracy_score(y_test, y_predict_m_nb)*100)
print('')
print('')

display(Markdown('**Random Forest Tree Classifier**'))
print(classification_report(y_test, y_predict_rf))
print("Accuracy is = ", accuracy_score(y_test, y_predict_rf)*100)
print('')
print('')

print('')
display(Markdown('**K-Nearest Neighbor Classifier**'))
print(classification_report(y_test, y_predict_KNN))
print("Accuracy is = ", accuracy_score(y_test, y_predict_KNN)*100)
print('')
print('')

display(Markdown('**Logistic Regression Classifier**'))
print(classification_report(y_test, y_predict_lr))
print("Accuracy is = ", accuracy_score(y_test, y_predict_lr)*100)
print('')
print('')

display(Markdown('**Decision Tree Classifier**'))
print(classification_report(y_test, y_predict_dtc))
print("Accuracy is = ", accuracy_score(y_test, y_predict_dtc)*100)

**Support Vector Machine Classifier**

              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71       248
         1.0       0.00      0.00      0.00       207

    accuracy                           0.55       455
   macro avg       0.27      0.50      0.35       455
weighted avg       0.30      0.55      0.38       455

Accuracy is =  54.505494505494504




/Users/banks007/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Naive Bayes - Multinomial Classifier**

              precision    recall  f1-score   support

         0.0       0.76      0.35      0.48       248
         1.0       0.53      0.87      0.66       207

    accuracy                           0.59       455
   macro avg       0.65      0.61      0.57       455
weighted avg       0.66      0.59      0.56       455

Accuracy is =  58.681318681318686




...

              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92       248
         1.0       0.98      0.82      0.89       207

    accuracy                           0.91       455
   macro avg       0.92      0.90      0.91       455
weighted avg       0.92      0.91      0.91       455

Accuracy is =  90.98901098901099




**Logistic Regression Classifier**

              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71       248
         1.0       1.00      0.01      0.02       207

    accuracy                           0.55       455
   macro avg       0.77      0.50      0.36       455
weighted avg       0.75      0.55      0.39       455

Accuracy is =  54.94505494505495




**Decision Tree Classifier**

              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71       248
         1.0       0.00      0.00      0.00       207

    accuracy                           0.55       455
   macro avg       0.27      0.50      0.35       455
weighted avg       0.30      0.55      0.38       455

Accuracy is =  54.505494505494504


In [79]:
get_score(KNeighborsClassifier(), x_train, x_test, y_train, y_test)

0.7186813186813187